<a href="https://colab.research.google.com/github/Joykw1/NLP_RAG_project/blob/main/Single_Passage_Retrieval_RAG_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open in Colab](https://colab.research.google.com/assets//colab-badge.svg)](https://colab.research.google.com/github/Joykw1/NLP_RAG_project/blob/main/Code/Single_Passage_Retrieval_RAG.ipynb)

In [1]:
# Downloads

!pip install datasets rank_bm25 bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# Imports
from datasets import load_dataset
from rank_bm25 import BM25Okapi
import random
from tqdm import tqdm

In [3]:
# Setup tokenizer. From tutorial 6

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

checkpoint = "Qwen/Qwen2.5-1.5B-Instruct"

# Configure 8-bit quantization. We use this to save VRAM, as we don't have a lot available.
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True  # Enables 8-bit quantization
)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    quantization_config=bnb_config,  # Apply BitsAndBytesConfig
    device_map="cuda"   # Assign to GPU
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
# Load the xquad dataset
dataset = load_dataset("xquad", "xquad.en")

# Print some info about the dataset
dataset

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/212k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/1190 [00:00<?, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1190
    })
})

In [ ]:
# This code is basically all from tutorial 6

contextCol = dataset['validation']['context']
contextCol = list(set(contextCol))
random.shuffle(contextCol) # randomize order of the list

questions = dataset['validation']['question']
answers = dataset['validation']['answers']

# print(len(questions))
# print(len(contextCol))

correct_count = 0

# Test single passage retrieval with 100 question examples
for i in tqdm(range(len(questions))):
  query = questions[i]

  # Tokenize documents for BM25
  tokenized_contexts = [doc.split() for doc in contextCol]
  bm25 = BM25Okapi(tokenized_contexts)

  # Retrieve top_k passages using BM25
  top_k = 1
  tokenized_query = query.split()
  top_docs = bm25.get_top_n(tokenized_query, contextCol, n=top_k)

  # Apply chat template
  context = "\n".join(top_docs)
  chat = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": f"Context: {context}\nQuery: {query}"},
  ]

  prompt = tokenizer.apply_chat_template(chat, tokenize=False)


  # Generate response
  input_ids = tokenizer.encode(prompt, return_tensors='pt').to("cuda")
  output_ids = model.generate(input_ids)
  response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
  if answers[i]['text'][0] in response:
    correct_count += 1
  else: print(f"\nWrong answer, correct answer was: {answers[i]['text'][0]},\nResponse:, {response}")

  #if i in [1,50,99]: print("\nResponse:", response)


print(correct_count, " correct out of ", i+1)

1190
240


  0%|          | 4/1190 [00:09<47:01,  2.38s/it]


Wrong answer, correct answer was: four,
Response:, system
You are a helpful assistant
user
Context: With 4:51 left in regulation, Carolina got the ball on their own 24-yard line with a chance to mount a game-winning drive, and soon faced 3rd-and-9. On the next play, Miller stripped the ball away from Newton, and after several players dove for it, it took a long bounce backwards and was recovered by Ward, who returned it five yards to the Panthers 4-yard line. Although several players dove into the pile to attempt to recover it, Newton did not and his lack of aggression later earned him heavy criticism. Meanwhile, Denver's offense was kept out of the end zone for three plays, but a holding penalty on cornerback Josh Norman gave the Broncos a new set of downs. Then Anderson scored on a 2-yard touchdown run and Manning completed a pass to Bennie Fowler for a 2-point conversion, giving Denver a 24–10 lead with 3:08 left and essentially putting the game away. Carolina had two more drives, 

  1%|          | 13/1190 [00:32<49:49,  2.54s/it]


Wrong answer, correct answer was: Luke Kuechly.,
Response:, system
You are a helpful assistant
user
Context: Doctor Who has appeared on stage numerous times. In the early 1970s, Trevor Martin played the role in Doctor Who and the Daleks in the Seven Keys to Doomsday. In the late 1980s, Jon Pertwee and Colin Baker both played the Doctor at different times during the run of a play titled Doctor Who – The Ultimate Adventure. For two performances, while Pertwee was ill, David Banks (better known for playing Cybermen) played the Doctor. Other original plays have been staged as amateur productions, with other actors playing the Doctor, while Terry Nation wrote The Curse of the Daleks, a stage play mounted in the late 1960s, but without the Doctor.
Query: Who was the Panthers' tackle leader for 2015?
system
I'm sorry, I couldn't find any information about a Panthers' tackle leader for

Check unable to find




...


Wrong answer, correct answer was: Major General James Abercrombie,
Response:, system
You are a helpful assistant
user
Context: Doctor Who has appeared on stage numerous times. In the early 1970s, Trevor Martin played the role in Doctor Who and the Daleks in the Seven Keys to Doomsday. In the late 1980s, Jon Pertwee and Colin Baker both played the Doctor at different times during the run of a play titled Doctor Who – The Ultimate Adventure. For two performances, while Pertwee was ill, David Banks (better known for playing Cybermen) played the Doctor. Other original plays have been staged as amateur productions, with other actors playing the Doctor, while Terry Nation wrote The Curse of the Daleks, a stage play mounted in the late 1960s, but without the Doctor.
Query: Who was appointed as second in command to Lor Loudoun in 1756?
system
I'm sorry, I couldn't find any information about "Lor Loudoun"

Check unable to find




100%|█████████▉| 1187/1190 [50:02<00:07,  2.58s/it]


Wrong answer, correct answer was: stress tensor,
Response:, system
You are a helpful assistant
user
Context: Child labour is common in Kenya. Most working children are active in agriculture. In 2006, UNICEF estimated that up to 30% of girls in the coastal areas of Malindi, Mombasa, Kilifi, and Diani were subject to prostitution. Most of the prostitutes in Kenya are aged 9–18. The Ministry of Gender and Child Affairs employed 400 child protection officers in 2009. The causes of child labour include poverty, the lack of access to education and weak government institutions. Kenya has ratified Convention No. 81 on labour inspection in industries and Convention No. 129 on labour inspection in agriculture.
Query: What causes strain in structures?
system
The query "What causes strain in structures?" is asking about the factors or conditions that


100%|██████████| 1190/1190 [50:10<00:00,  2.53s/it]

939  correct out of  1190
From these, the model could not find 64 answers from the context.


### Result:
939  correct out of  1190

In [4]:
# Load the xquad dataset
dataset = load_dataset("xquad", "xquad.de")

# Print some info about the dataset
dataset

README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/242k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/1190 [00:00<?, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1190
    })
})

In [6]:
# This code is basically all from tutorial 6

contextCol = dataset['validation']['context']
contextCol = list(set(contextCol))
random.shuffle(contextCol) # randomize order of the list

questions = dataset['validation']['question']
answers = dataset['validation']['answers']

# print(len(questions))
# print(len(contextCol))

correct_count = 0


# Test single passage retrieval with 100 question examples
for i in tqdm(range(len(questions))):
  query = questions[i]

  # Tokenize documents for BM25
  tokenized_contexts = [doc.split() for doc in contextCol]
  bm25 = BM25Okapi(tokenized_contexts)

  # Retrieve top_k passages using BM25
  top_k = 1
  tokenized_query = query.split()
  top_docs = bm25.get_top_n(tokenized_query, contextCol, n=top_k)

  # Apply chat template
  context = "\n".join(top_docs)
  chat = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": f"Context: {context}\nQuery: {query}"},
  ]

  prompt = tokenizer.apply_chat_template(chat, tokenize=False)

  #if i in [1,50,99]: print("\nPrompt:", prompt)

  # Generate response
  input_ids = tokenizer.encode(prompt, return_tensors='pt').to("cuda")
  output_ids = model.generate(input_ids)
  response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
  if answers[i]['text'][0] in response:
    correct_count += 1
  else: print(f"\nWrong answer, correct answer was: {answers[i]['text'][0]},\nResponse:, {response}")
  #if i in [1,50,99]: print("\nResponse:", response)


print(f"\n {correct_count} correct out of {i+1}")

1190
240


  0%|          | 4/1190 [00:12<58:34,  2.96s/it]  


Wrong answer, correct answer was: vier,
Response:, system
You are a helpful assistant
user
Context: Mit 4:51 verbleibenden Minuten in der regulären Spielzeit brachte Carolina den Ball auf die eigene 24-Yard Line und bekam die Chance, einen spielentscheidenden Drive zu erzielen. Bald darauf standen sie einem 3rd-and-9 gegenüber. Beim nächsten Spielzug nahm Miller Newton den Ball ab und nach mehreren Dives flog der Ball eine weite Strecke nach hinten und wurde von Ward erobert, der ihn fünf Yard auf die 4-Yard Line der Panthers zurückbrachte. Obwohl mehrere Spieler in den Pile sprangen, um zu versuchen, den Ball zurückzugewinnen, tat Newton dies nicht. Sein Mangel an Aggression brachte ihm später heftige Kritik ein. In der Zwischenzeit wurde Denvers Offensive für drei Spielzüge aus der Endzone herausgehalten, aber eine Holding-Strafe für Cornerback Josh Norman gab den Broncos ein neues Set Downs. Dann punktete Anderson bei einem 2-Yard-Touchdown-Run und Manning vollendete einen Pass zu 

  1%|          | 10/1190 [00:28<50:49,  2.58s/it]


Wrong answer, correct answer was: Kurt Coleman,
Response:, system
You are a helpful assistant
user
Context: Mit 4:51 verbleibenden Minuten in der regulären Spielzeit brachte Carolina den Ball auf die eigene 24-Yard Line und bekam die Chance, einen spielentscheidenden Drive zu erzielen. Bald darauf standen sie einem 3rd-and-9 gegenüber. Beim nächsten Spielzug nahm Miller Newton den Ball ab und nach mehreren Dives flog der Ball eine weite Strecke nach hinten und wurde von Ward erobert, der ihn fünf Yard auf die 4-Yard Line der Panthers zurückbrachte. Obwohl mehrere Spieler in den Pile sprangen, um zu versuchen, den Ball zurückzugewinnen, tat Newton dies nicht. Sein Mangel an Aggression brachte ihm später heftige Kritik ein. In der Zwischenzeit wurde Denvers Offensive für drei Spielzüge aus der Endzone herausgehalten, aber eine Holding-Strafe für Cornerback Josh Norman gab den Broncos ein neues Set Downs. Dann punktete Anderson bei einem 2-Yard-Touchdown-Run und Manning vollendete einen 

...


Wrong answer, correct answer was: Spannungstensor,
Response:, system
You are a helpful assistant
user
Context: Der erste methodistische Klerus wurde von John Wesley, einem Priester der Church of England, ordiniert in Anbetracht der Krise, die durch die amerikanische Revolution verursacht wurde, die die Methodisten in den Staaten von der Church of England und ihren Sakramenten isolierte. Heutzutage umfasst der Klerus Männer und Frauen, die von den Bischöfen als Älteste und den Diakonen ordiniert und in verschiedene Ämter berufen werden. Die Ältesten in der Evangelisch-methodistischen Kirche verehren die und unterliegen der Autorität und Ernennung ihrer Bischöfe. Sie dienen in der Regel als Pastoren in den Ortsgemeinden. Die Diakone stehen im Dienst der Kirche und können als Musiker, Liturgen, Erzieher, Betriebswirte und in einer Reihe anderer Bereiche tätig sein. Älteste und Diakone müssen einen Master-Abschluss (in der Regel einen M.Div.) oder einen anderen gleichwertigen Abschluss vo

100%|█████████▉| 1188/1190 [52:31<00:05,  2.75s/it]


Wrong answer, correct answer was: Druckkräfte,
Response:, system
You are a helpful assistant
user
Context: Seitdem und bis heute gilt die allgemeine Relativitätstheorie als die Theorie, die die Schwerkraft am besten erklärt. In der ART wird die Gravitation nicht als Kraft betrachtet, sondern dass die Objekte, die sich in Gravitationsfeldern frei bewegen, unter ihrer eigenen Trägheit in geraden Linien durch die gekrümmte Raumzeit wandern – definiert als der kürzeste Raum-Zeit-Weg zwischen zwei Raum-Zeit-Ereignissen. Aus der Perspektive des Objekts geschieht jede Bewegung, als gäbe es überhaupt keine Gravitation. Nur wenn man die Bewegung in einem globalen Sinne betrachtet, kann man die Krümmung der Raumzeit beobachten und die Kraft aus dem Kurvenverlauf des Objekts ableiten. So wird der gerade Weg im Raum-Zeit-Kontinuum als eine gekrümmte Linie im Raum gesehen und als ballistische Trajektorie des Objekts bezeichnet. Ein am Boden geworfener Basketball fliegt in einer Parabel, da er sich

100%|██████████| 1190/1190 [52:37<00:00,  2.65s/it]


Wrong answer, correct answer was: Formalismus,
Response:, system
You are a helpful assistant
user
Context: Das nützlichste Messgerät zur Analyse der Leistung von Dampfmaschinen ist der Dampfmaschinen-Indikator. Frühe Versionen waren ab 1851 in Gebrauch, aber der erfolgreichste Indikator wurde für den Erfinder von Hochgeschwindigkeitsmaschinen, Charles Porter, von Charles Richard entwickelt und 1862 bei der London Exhibition präsentiert. Der Dampfmaschinen-Indikator zeichnet auf einem Papier den Druck im Zylinder über den Zyklus auf, was zum Aufspüren verschiedener Probleme und zur Berechnung der entwickelten Pferdestärken genutzt werden kann. Er wurde von Ingenieuren, Mechanikern und Versicherungsinspektoren routinemäßig eingesetzt. Der Maschinen-Indikator kann auch bei Verbrennungsmotoren verwendet werden. Siehe unten die Abbildung eines Indikatordiagramms (im Abschnitt zu den Motorarten).
Query: Was beinhaltet Druckkräfte bei der Berechnung der Fläche im Volumen?
user
Die Berechnung

### Result:
832 correct out of 1190

In [7]:
# Load the xquad dataset
dataset = load_dataset("xquad", "xquad.ru")

# Print some info about the dataset
dataset

validation-00000-of-00001.parquet:   0%|          | 0.00/322k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/1190 [00:00<?, ? examples/s]

DatasetDict({
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1190
    })
})

In [8]:
# This code is basically all from tutorial 6

contextCol = dataset['validation']['context']
contextCol = list(set(contextCol))
random.shuffle(contextCol) # randomize order of the list

questions = dataset['validation']['question']
answers = dataset['validation']['answers']

# print(len(questions))
# print(len(contextCol))

correct_count = 0

# Test single passage retrieval with 100 question examples
for i in tqdm(range(len(questions))):
  query = questions[i]

  # Tokenize documents for BM25
  tokenized_contexts = [doc.split() for doc in contextCol]
  bm25 = BM25Okapi(tokenized_contexts)

  # Retrieve top_k passages using BM25
  top_k = 1
  tokenized_query = query.split()
  top_docs = bm25.get_top_n(tokenized_query, contextCol, n=top_k)

  # Apply chat template
  context = "\n".join(top_docs)
  chat = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": f"Context: {context}\nQuery: {query}"},
  ]

  prompt = tokenizer.apply_chat_template(chat, tokenize=False)

  #if i in [1,50,99]: print("\nPrompt:", prompt)

  # Generate response
  input_ids = tokenizer.encode(prompt, return_tensors='pt').to("cuda")
  output_ids = model.generate(input_ids)
  response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
  if answers[i]['text'][0] in response:
    correct_count += 1
  else: print(f"\nWrong answer, correct answer was: {answers[i]['text'][0]},\nResponse:, {response}")
  #if i in [1,50,99]: print("\nResponse:", response)

#print(idx_admin)

print(f"\n {correct_count} correct out of {i+1}")

1190
240


  0%|          | 1/1190 [00:04<1:26:49,  4.38s/it]


Wrong answer, correct answer was: 308,
Response:, system
You are a helpful assistant
user
Context: Бронкос победил Питтсбург Стилерс в дивизионном раунде, 23–16, набрав 11 очков в последние три минуты игры. Затем они победили действующего чемпиона Суперкубка XLIX Нью-Ингленд Пэтриотс в игре чемпионата АФК, 20–18, перехватив пас на попытку двухочковой конверсии Нью-Ингленд, когда на часах оставалось 17 секунд. Несмотря на проблемы Мэннинга с перехватами в течение сезона, он не проиграл ни одной в их двух игр плей-офф.
Query: Сколько очков уступила защита Пэнтерс?
 superintendent
The defense of the Patriots lost 14 points in total to the Broncos.


  0%|          | 3/1190 [00:10<1:06:43,  3.37s/it]


Wrong answer, correct answer was: 118,
Response:, system
You are a helpful assistant
user
Context: Взаимосвязь между капитализмом, аристократией и империализмом долго являлась предметом споров среди историков и политических мыслителей. Первыми теоретиками, занявшимися обсуждением этой темы, были Дж. А. Гобсон (1858–1940), Йозеф Шумпетер (1883–1950), Торстейн Веблен (1857–1929) и Норман Энджелл (1872–1967). Хотя наиболее плодотворный период этих мыслителей немарксистского толка приходится на время перед Первой мировой войной, они также продолжали писать и в период между войнами. Их работа в совокупности обеспечила основу для изучения империализма и его влияния на Европу, а также была полезна для осмысления подъема военно-политического комплекса США, начиная с 1950-х годов. Гобсон был убежден в том, что внутренние социальные реформы могли бы излечить международную болезнь империализма, подорвав саму ее экономическую основу. Согласно его теории, государственное вмешательство за счет нало

...


Wrong answer, correct answer was: Тензор напряжений,
Response:, system
You are a helpful assistant
user
Context: В противоположность этому, в периоды бодрствования количество дифференцированных эффекторных клеток, таких как цитотоксические естественные клетки-киллеры и клетки CTL (цитотоксические Т-лимфоциты), достигает максимума, и это вызывает эффективную реакцию, направленную против каких бы то ни было проникающих возбудителей заболеваний. Также в периоды активного бодрствования количество противовоспалительных молекул, таких как кортизол и катехоламин, достигает максимума. Имеются две теории, объясняющие, почему провоспалительное состояние бывает во время сна. Во-первых, воспаление вызывало бы серьезные когнитивные и соматические нарушения, если бы оно происходило в периоды бодрствования. Во-вторых, воспаление может происходить во время сна из-за присутствия мелатонина. Воспаление вызывает сильный окислительный стресс, и присутствие мелатонина во время сна могло бы активно препятс

100%|█████████▉| 1189/1190 [51:52<00:02,  2.74s/it]


Wrong answer, correct answer was: компоненты давления,
Response:, system
You are a helpful assistant
user
Context: В 1993 году Галор и Зеира показали, что неравенство в присутствии несовершенств кредитного рынка имеет длительное пагубное влияние на формирование человеческого капитала и экономическое развитие. В исследовании от 1996 года Перотти рассмотрел каналы, через которые неравенство может влиять на экономический рост. Он показал, что, в соответствии с подходом несовершенств кредитного рынка, неравенство связано с более низким уровнем формирования человеческого капитала (образование, опыт и обучение) и более высоким уровнем рождаемости, и, следовательно, с более низкими уровнями роста. Он обнаружил, что неравенство связано с более высокими уровнями перераспределительного налогообложения, что связано с более низкими уровнями роста в результате сокращения частных сбережений и инвестиций. Перотти пришел к выводу, что "более равные общества имеют более низкие показатели рождаемости и

100%|██████████| 1190/1190 [51:54<00:00,  2.62s/it]


 755, correct out of  1190
From these, the model could not find 8 answers from the context.


### Result:
755 correct out of 1190